In [15]:
def predict(x):
    
    from pyspark.sql.functions import substring,conv,col,min,max,udf,lit
   
    d1 = spark.read.csv('/user/maxnet/database/sig.db/term_desc_all',sep='\x01')
    d1 = d1.withColumnRenamed('_c0','term_id')\
           .withColumnRenamed('_c1','brand')\
           .withColumnRenamed('_c2','cn_name')\
           .withColumnRenamed('_c3','en_name')\
           .withColumnRenamed('_c4','type1')\
           .withColumnRenamed('_c5','type2')\
           .withColumnRenamed('_c6','os')\
           .withColumnRenamed('_c7','dtime')\
           .withColumnRenamed('_c8','price')\
           .withColumnRenamed('_c9','remarks')\
           .withColumnRenamed('_c10','prio')

    d2 = spark.read.csv('/user/maxnet/database/sig.db/term_oui',sep='\x01')
    d2 = d2.withColumnRenamed('_c0','mac_6').withColumnRenamed('_c1','manu')
    
    d = spark.read.csv('/user/maxnet/database/sig.db/data_mac_res',sep='\x01')
    d = d.withColumnRenamed('_c0','mac').withColumnRenamed('_c1','id')
    d = d.withColumn('m1_6',substring(d.mac,1,6))

    tmp_1 = d.join(d2,d.m1_6==d2.mac_6)
    df = tmp_1.join(d1,tmp_1.id == d1.term_id).select('mac','id','manu','brand','prio').distinct().dropna()
    df = df.filter(df.prio == 4).select('mac','id','brand').distinct().dropna()
    
    df = df.withColumn('m1',substring(df.mac,1,6)).withColumn('m2',substring(df.mac,7,6))
    df = df.withColumn('p',conv(df.m2, 16, 10))
    df = df.select('mac','id','m1','m2',col('p').cast('bigint'))
    d3 = df.filter(df.m1 =='1C5CF2')
    
    bottom = d3.select(min('p')).collect()[0]['min(p)']
    top = d3.select(max('p')).collect()[0]['max(p)'] 
    
    if x < bottom:
        return d3.filter(d3.p == bottom).select('id').collect()[0]['p']
    
    elif x > top:
        return d3.filter(d3.p == top).select('id').collect()[0]['p']
    
    else :
        tmp1 = d3.where(col('p').between((x-10),(x+10))).groupBy('id').count()
        tmp2 = d3.where(col('p').between((x-100),(x+100))).groupBy('id').count()
        tmp3 = d3.where(col('p').between((x-1000),(x+1000))).groupBy('id').count()
        tmp4 = d3.where(col('p').between((x-10000),(x+10000))).groupBy('id').count()
        tmp5 = d3.where(col('p').between((x-100000),(x+100000))).groupBy('id').count()
        return tmp1.union(tmp2).union(tmp3).union(tmp4).union(tmp5).groupBy('id').count().sort('count',ascending=False).collect()[0]['id'] 

In [16]:
from pyspark.sql.functions import substring,conv,col,min,max,udf,lit
   
d1 = spark.read.csv('/user/maxnet/database/sig.db/term_desc_all',sep='\x01')
d1 = d1.withColumnRenamed('_c0','term_id')\
       .withColumnRenamed('_c1','brand')\
       .withColumnRenamed('_c2','cn_name')\
       .withColumnRenamed('_c3','en_name')\
       .withColumnRenamed('_c4','type1')\
       .withColumnRenamed('_c5','type2')\
       .withColumnRenamed('_c6','os')\
       .withColumnRenamed('_c7','dtime')\
       .withColumnRenamed('_c8','price')\
       .withColumnRenamed('_c9','remarks')\
       .withColumnRenamed('_c10','prio')

d2 = spark.read.csv('/user/maxnet/database/sig.db/term_oui',sep='\x01')
d2 = d2.withColumnRenamed('_c0','mac_6').withColumnRenamed('_c1','manu')

d = spark.read.csv('/user/maxnet/database/sig.db/data_mac_res',sep='\x01')
d = d.withColumnRenamed('_c0','mac').withColumnRenamed('_c1','id')
d = d.withColumn('m1_6',substring(d.mac,1,6))

tmp_1 = d.join(d2,d.m1_6==d2.mac_6)
df = tmp_1.join(d1,tmp_1.id == d1.term_id).select('mac','id','manu','brand','prio').distinct().dropna()
df = df.filter(df.prio == 4).select('mac','id','brand').distinct().dropna()

df = df.withColumn('m1',substring(df.mac,1,6)).withColumn('m2',substring(df.mac,7,6))
df = df.withColumn('p',conv(df.m2, 16, 10))
df = df.select('mac','id','m1','m2',col('p').cast('bigint'))
d3 = df.filter(df.m1 =='1C5CF2')

In [17]:
d3.sample(0.001).show()

+------------+-----+------+------+--------+
|         mac|   id|    m1|    m2|       p|
+------------+-----+------+------+--------+
|1C5CF2C9B22E|49809|1C5CF2|C9B22E|13218350|
|1C5CF2BBA078|49809|1C5CF2|BBA078|12296312|
|1C5CF2F2CE6F|49809|1C5CF2|F2CE6F|15912559|
|1C5CF28FB2D3|49809|1C5CF2|8FB2D3| 9417427|
|1C5CF23558F7|49809|1C5CF2|3558F7| 3496183|
|1C5CF2C71743|49809|1C5CF2|C71743|13047619|
|1C5CF229107A|49809|1C5CF2|29107A| 2691194|
|1C5CF284CA34|49809|1C5CF2|84CA34| 8702516|
|1C5CF233F9F2|49809|1C5CF2|33F9F2| 3406322|
|1C5CF26AF037|49809|1C5CF2|6AF037| 7008311|
|1C5CF26FB3F6|49809|1C5CF2|6FB3F6| 7320566|
|1C5CF2142244|49809|1C5CF2|142244| 1319492|
|1C5CF2AD8FD8|49809|1C5CF2|AD8FD8|11374552|
|1C5CF2EC6255|49809|1C5CF2|EC6255|15491669|
|1C5CF21E6E92|49810|1C5CF2|1E6E92| 1994386|
|1C5CF2BB6DCE|49809|1C5CF2|BB6DCE|12283342|
|1C5CF2CE3202|49809|1C5CF2|CE3202|13513218|
|1C5CF2F267FA|49809|1C5CF2|F267FA|15886330|
|1C5CF2304F6B|49809|1C5CF2|304F6B| 3166059|
|1C5CF2CB6BB3|49809|1C5CF2|CB6BB

In [26]:
predict(1994200)

'49809'

In [27]:
d3.filter(d3.p == 1994200 ).show()

+---+---+---+---+---+
|mac| id| m1| m2|  p|
+---+---+---+---+---+
+---+---+---+---+---+



In [15]:
#import pandas as pd
#test = spark.read.parquet('/data/user/hive/warehouse/ian/test')
#result = []
#for x in test.select('p').toPandas()['p'].values.tolist():
#    result.append(predict(x))
    
#mac = test.select('mac').toPandas()['mac'].values.tolist()
#pre = spark.createDataFrame(pd.DataFrame({'mac':mac,'predict_id':result}))

In [17]:
#pre = pre.withColumnRenamed('mac','mac1')
#dd = test.join(pre,test.mac == pre.mac1,how='left')

In [18]:
#dd.show(50)

+------------+-----+------+------+--------+------------+----------+
|         mac|   id|    m1|    m2|       p|        mac1|predict_id|
+------------+-----+------+------+--------+------------+----------+
|702ED911A749|42980|702ED9|11A749| 1156937|702ED911A749|     42980|
|702ED9EFA037|10666|702ED9|EFA037|15704119|702ED9EFA037|     10666|
|702ED93624FC|42980|702ED9|3624FC| 3548412|702ED93624FC|     42980|
|702ED908DF30|42980|702ED9|08DF30|  581424|702ED908DF30|     42980|
|702ED91FBD70|42980|702ED9|1FBD70| 2080112|702ED91FBD70|     42980|
|702ED934C76D|42980|702ED9|34C76D| 3458925|702ED934C76D|     42980|
|702ED906FC7A|42980|702ED9|06FC7A|  457850|702ED906FC7A|     42980|
|702ED9247293|42980|702ED9|247293| 2388627|702ED9247293|     42980|
|702ED934B44D|42980|702ED9|34B44D| 3454029|702ED934B44D|     42980|
|702ED90CC220|42980|702ED9|0CC220|  836128|702ED90CC220|     42980|
|702ED9BCEAE0|41347|702ED9|BCEAE0|12380896|702ED9BCEAE0|     41347|
|702ED925F017|42980|702ED9|25F017| 2486295|702ED